In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split

In [1]:
!pip install optuna


[notice] A new release of pip available: 22.2.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
class N_grams:
    
    def __init__(self, url):
        self.url = url
        self.truthful_unigram_txt = {}
        self.truthful_txt = None
        self.truthful_unigram_dict = {}
        self.total_words_count = None
        self.bigram_dictionary_pairs = {}
        self.total_words_count_bigram = None
        self.bigram_probability_pairs = {}
        self.wordslist_truthful = []
        self.vocab_size_truthful = None
        self.add_one_matrix_truthful = None
        self.row_names_truthful = None
        self.column_names_truthful = None
        self.smc = None
        self.truthful_unigram_dict_filtered = {}
        self.truthful_unigram_dict_filtered_final  = {}
        self.truthful_bigram_dict_filtered = {}
        self.truthful_bigram_dict_filtered_final = {}
        self.truth_perplexity_unigram = 0
        self.truth_perplexity_bigram = 0
        
        
        
    def DataProcessing(self):
        self.truthful_txt = pd.read_csv(self.url, sep = "\t", header = None)
        #print(self.truthful_txt.head())
        #rint(len(self.truthful_txt))
    
    def process_word(self, word):
        if word.isalpha():
            if word not in self.truthful_unigram_txt:
                self.truthful_unigram_txt[word] = 1
            else:
                self.truthful_unigram_txt[word] = self.truthful_unigram_txt[word] + 1

    def build_unigram_dict(self, text):
        for row in range(len(text)):
            for word in text[0][row].split():
                self.process_word(word)

    def UnigramDict(self):
        self.build_unigram_dict(self.truthful_txt)

        print(self.truthful_unigram_txt)
        self.total_words_count = sum(self.truthful_unigram_txt.values())

        return self.total_words_count

    
    def calculate_unigram_probabilities(self, unigram_counts):
        unigram_probabilities = {word: count / self.total_words_count for word, count in unigram_counts.items()}
        return unigram_probabilities

    def Probability_Dict_Unigram(self):
        unigram_probabilities = self.calculate_unigram_probabilities(self.truthful_unigram_txt)

        # You can print the probabilities or perform other actions here if needed
        # print(unigram_probabilities)

        return unigram_probabilities

        
    def Validation(self):
        self.validation_truthful = pd.read_csv("https://raw.githubusercontent.com/YaswanthAd/n_gram_lm/main/val.txt", sep="\t", header=None)
        
        
    def BigramDict(self):
        
        for row in range(len(self.truthful_txt)):
            row_as_list = self.truthful_txt[0][row].split()
            row_as_list.insert(0,"START")
            row_as_list.append("END")
            row_as_list = [values for values in row_as_list if values.isalpha()]

            for i in range(len(row_as_list)-1):
                if row_as_list[i].isalpha():
                    if (row_as_list[i],row_as_list[i+1]) not in self.bigram_dictionary_pairs.keys():
                        self.bigram_dictionary_pairs[(row_as_list[i],row_as_list[i+1])]=1
                    else:
                        self.bigram_dictionary_pairs[(row_as_list[i],row_as_list[i+1])]+=1
                        
        # print(self.bigram_dictionary_pairs)
        self.total_words_count_bigram = sum(self.bigram_dictionary_pairs.values())
        
        return self.total_words_count_bigram
        
    def Probability_Dict_Bigram(self):
        
        for key,val in self.bigram_dictionary_pairs.items():
            if key[0] == "START":
                self.bigram_probability_pairs[key] = val/len(self.truthful_txt)
            else:
                self.bigram_probability_pairs[key] = val/self.truthful_unigram_txt[key[0]]
                
        # print(self.bigram_probability_pairs)
        
    def perplexity_unigram_unk(self, sentence_as_list,unigram_dictionary):
        prob_sum,length_of_sentence = 0,len(sentence_as_list)
        for word in sentence_as_list:
            if word in unigram_dictionary.keys():
                prob_sum += (math.log(unigram_dictionary[word],2))
            else:
                prob_sum += (math.log(unigram_dictionary["UNK"],2))
        l = (prob_sum/length_of_sentence)
        return 2**-l
    
    def perplexity_bigram_unk(self,  sentence_as_list,bigram_dictionary):
        prob_sum,length_of_sentence = 0,len(sentence_as_list)
        for i in range(len(sentence_as_list)-1):
            if (sentence_as_list[i],sentence_as_list[i+1]) in bigram_dictionary.keys():
                prob_sum += (math.log(bigram_dictionary[(sentence_as_list[i],sentence_as_list[i+1])],2))
            else:
                prob_sum += (math.log(bigram_dictionary[("UNK","UNK")],2))
        l = (prob_sum/length_of_sentence)
        return 2**-l
    
    def perplexity_bigram_matrix(self, sentence_as_list,bigram_matrix):
        prob_sum,length_of_sentence = 0,len(sentence_as_list)
        for i in range(len(sentence_as_list)-1):
            if sentence_as_list[i] in bigram_matrix.index and sentence_as_list[i] != "END" and sentence_as_list[i+1] in bigram_matrix.columns and sentence_as_list[i+1] != "START":
                prob_sum += (math.log(bigram_matrix.at[sentence_as_list[i],sentence_as_list[i+1]],2))
        l = (prob_sum/length_of_sentence)
        return 2**-l
    
    def smoothing_matrix(self):
        
        for key in self.truthful_unigram_txt.keys():
            self.wordslist_truthful.append(key)
            
        self.wordslist_truthful.insert(0,"START")
        self.wordslist_truthful.append("END")
        self.vocab_size_truthful = len(self.wordslist_truthful)
        self.row_names_truthful,self.column_names_truthful = self.wordslist_truthful,self.wordslist_truthful
        matrix = np.zeros((self.vocab_size_truthful, self.vocab_size_truthful))
        self.add_one_matrix_truthful = pd.DataFrame(matrix, columns=self.column_names_truthful, index=self.row_names_truthful)
       # print(self.add_one_matrix_truthful.head())
    
    def one_smoothing_matrix(self, k):
        
        self.smc = k
        for pair in self.bigram_dictionary_pairs.keys():
            if pair[0] == "START":
                self.add_one_matrix_truthful.at[pair[0],pair[1]] = (self.bigram_dictionary_pairs[pair]+(k))/(len(self.truthful_txt)+self.vocab_size_truthful)
            else:
                self.add_one_matrix_truthful.at[pair[0],pair[1]] = (self.bigram_dictionary_pairs[pair]+(k))/(self.truthful_unigram_txt[pair[0]]+self.vocab_size_truthful)
                
        for i in self.add_one_matrix_truthful.index:
            for j in self.add_one_matrix_truthful.columns:
                if self.add_one_matrix_truthful.at[i,j] == 0 and i!="END" and j!="START":
                    if i == "START":
                        self.add_one_matrix_truthful.at[i,j] = (k)/(len(self.truthful_txt)+self.vocab_size_truthful)
                    else:
                        self.add_one_matrix_truthful.at[i,j] = (k)/(self.truthful_unigram_txt[i]+self.vocab_size_truthful)
    
    
    def Perplexity_calculate(self):
    
        self.truth_perplexity_bigram = 0
        for row in range(len(self.validation_truthful)):
            row_as_list = self.validation_truthful[0][row].split()
            row_as_list = [values for values in row_as_list if values.isalpha()]
            self.truth_perplexity_bigram += self.perplexity_bigram_matrix(row_as_list,self.add_one_matrix_truthful)
    
        print("Total perplexity of the validation dataset using the "+ str(self.smc) + "smoothing model is:" + str(self.truth_perplexity_bigram))
        
        
    def UNK_handling_unigram(self):
        
        truthful_unigram_dict_filtered = {k: v for k, v in sorted(self.truthful_unigram_dict.items(), key=lambda item: item[1], reverse=True)}
        truthful_unigram_dict_filtered = {k: v for k,v in truthful_unigram_dict_filtered.items() if v>=45}
        print(len(truthful_unigram_dict_filtered))
        self.truthful_unigram_dict_filtered["UNK"] = sum(self.truthful_unigram_dict.values())-sum(truthful_unigram_dict_filtered.values())
        print(self.truthful_unigram_dict_filtered["UNK"])
        self.truthful_unigram_dict_filtered_final = {key:val/sum(truthful_unigram_dict_filtered.values()) for key,val in self.truthful_unigram_dict_filtered.items()}
        print(self.truthful_unigram_dict_filtered_final)
        
    def UNK_handling_bigram(self):
        
        self.truthful_bigram_dict_filtered = {k: v for k, v in sorted(self.bigram_dictionary_pairs.items(), key=lambda item: item[1], reverse=True)}
        self.truthful_bigram_dict_filtered = {k: v for k,v in self.truthful_bigram_dict_filtered.items() if v>=45}
        print(len(self.truthful_bigram_dict_filtered))
        self.truthful_bigram_dict_filtered[("UNK","UNK")] = sum(self.bigram_dictionary_pairs.values())-sum(self.truthful_bigram_dict_filtered.values())
        print(self.truthful_bigram_dict_filtered[("UNK","UNK")])
        self.truthful_bigram_dict_filtered_final = {key:val/sum(self.truthful_bigram_dict_filtered.values()) for key,val in self.truthful_bigram_dict_filtered.items()}
        print(self.truthful_bigram_dict_filtered_final)
        
    def perplexity_unigram_bigram_final(self):
    
        self.truth_perplexity_bigram = 0
        self.truth_perplexity_unigram = 0
        for row in range(len(self.validation_truthful)):
            row_as_list = self.validation_truthful[0][row].split()
            row_as_list = [values for values in row_as_list if values.isalpha()]
            self.truth_perplexity_unigram += self.perplexity_unigram_unk(row_as_list,self.truthful_unigram_dict_filtered_final)
            self.truth_perplexity_bigram += self.perplexity_bigram_unk(row_as_list,self.truthful_bigram_dict_filtered_final)
        print("Total perplexity of the truthful validation dataset using the unigram model is: " + str(self.truth_perplexity_unigram))
        print("Total perplexity of the truthful validation dataset using the Bigram model is: " + str(self.truth_perplexity_bigram))

        

In [3]:
if __name__ == "__main__":
    ng = N_grams("https://raw.githubusercontent.com/YaswanthAd/n_gram_lm/main/train.txt")
    ng.DataProcessing()
    total_words = ng.UnigramDict()
    print(total_words)
    ng.Probability_Dict_Unigram()
    total_words_bigram = ng.BigramDict()
    print(total_words_bigram)
    ng.Probability_Dict_Bigram()

{'I': 1659, 'booked': 81, 'two': 120, 'rooms': 183, 'four': 11, 'months': 7, 'in': 1212, 'advance': 7, 'at': 717, 'the': 4250, 'Talbott': 28, 'We': 461, 'were': 578, 'placed': 8, 'on': 610, 'top': 38, 'floor': 135, 'next': 99, 'to': 2077, 'elevators': 31, 'which': 173, 'are': 308, 'used': 31, 'all': 245, 'night': 198, 'long': 37, 'When': 88, 'speaking': 3, 'front': 121, 'desk': 155, 'was': 1820, 'told': 77, 'that': 662, 'they': 380, 'simply': 17, 'honoring': 1, 'my': 419, 'request': 21, 'for': 997, 'an': 219, 'upper': 5, 'had': 525, 'requested': 32, 'a': 2215, 'better': 89, 'view': 101, 'am': 62, 'looking': 43, 'brick': 6, 'wall': 30, 'and': 2552, 'getting': 28, 'no': 212, 'sleep': 28, 'He': 21, 'also': 105, 'me': 242, 'received': 19, 'complaints': 12, 'before': 61, 'from': 348, 'guests': 52, 'aware': 9, 'of': 1040, 'noise': 29, 'problem': 38, 'Why': 6, 'then': 59, 'did': 242, 'place': 85, 'us': 212, 'this': 453, 'when': 185, 'hotel': 923, 'is': 835, 'not': 570, 'totally': 8, 'A': 31, 

ng.wordlist()

In [ ]:
# import optuna

# def objective(trial):
#     # Define the range of values for k that you want to search over
#     k = trial.suggest_float('k', 0.005, 1.0)
    
#     # Perform the necessary steps with the chosen k value
#     ng.Validation()
#     ng.smoothing_matrix()
#     ng.one_smoothing_matrix(k=k)
    
#     # Calculate perplexity or another performance metric
#     perplexity = ng.Perplexity_calculate()
    
#     # Return the metric to be minimized (lower is better)
#     return perplexity

# # Create an Optuna study and optimize the objective function
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=5)  # You can adjust the number of trials as needed

In [4]:
ng.Validation()
ng.smoothing_matrix()
ng.one_smoothing_matrix(0.001)
# ng.Perplexity_calculate()

In [50]:
ng.Perplexity_calculate()

Total perplexity of the validation dataset using the 1smoothing model is:55573.0397336743


In [51]:
ng.UNK_handling_unigram()
ng.UNK_handling_bigram()
ng.perplexity_unigram_bigram_final()

0
1.0000000000001077
{'UNK': 1.0}
95
68313
{('in', 'the'): 0.0050733968580994075, ('of', 'the'): 0.00440381148596446, ('at', 'the'): 0.004236415142930724, ('the', 'hotel'): 0.0036955961885140358, ('and', 'the'): 0.003450939994849343, ('to', 'the'): 0.0033865567859902137, ('on', 'the'): 0.0027427246973989183, ('the', 'room'): 0.0025495750708215297, ('I', 'was'): 0.002356425444244141, ('for', 'a'): 0.002356425444244141, ('room', 'was'): 0.0023049188771568376, ('this', 'hotel'): 0.0021632758176667525, ('it', 'was'): 0.0019443729075457122, ('for', 'the'): 0.0018928663404584084, ('was', 'very'): 0.0018799896986865826, ('from', 'the'): 0.0017769765645119753, ('and', 'I'): 0.0017383466391964976, ('was', 'a'): 0.0017254699974246716, ('had', 'a'): 0.001661086788565542, ('with', 'a'): 0.0016224568632500643, ('a', 'great'): 0.001532320370847283, ('we', 'were'): 0.0014936904455318052, ('in', 'a'): 0.0014936904455318052, ('to', 'be'): 0.0014808138037599795, ('I', 'had'): 0.0014550605202163277, ('is